In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from transformers import AutoModelForMaskedLM, AutoTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def get_nt_embeddings(dataloader, tokenizer, model, pooling="mean"):
    embeddings = []
    labels = []

    model.to(device)
    model.eval()

    for batch in tqdm(dataloader, desc="Generating embeddings"):
        sequences, lbls = batch

        tokens = tokenizer(
            list(sequences), padding=True, truncation=True, max_length=512, return_tensors="pt"
        )
        input_ids = tokens["input_ids"].to(device)
        attention_mask = tokens["attention_mask"].to(device)

        with torch.no_grad():
            with torch.amp.autocast("cuda"):
                outputs = model(
                    input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True
                )

            last_hidden = outputs.hidden_states[-1]

            if pooling == "mean":
                attention_mask_exp = attention_mask.unsqueeze(-1)
                sum_embeddings = torch.sum(last_hidden * attention_mask_exp, dim=1)
                sum_mask = attention_mask_exp.sum(dim=1).clamp(min=1e-9)
                pooled = sum_embeddings / sum_mask
            elif pooling == "max":
                pooled = last_hidden.masked_fill(attention_mask.unsqueeze(-1) == 0, -1e9)
                pooled = torch.max(pooled, dim=1).values
            else:
                raise ValueError("pooling must be 'mean' or 'max'")

        embeddings.append(pooled.cpu().numpy())
        labels.append(lbls.numpy())

    return np.vstack(embeddings), np.concatenate(labels)

In [6]:
MODEL_NAME = "InstaDeepAI/nucleotide-transformer-v2-50m-multi-species"
PROCESSED_DIR = Path.cwd().resolve().parent / "data" / "processed"
EMB_DIR = Path.cwd().resolve().parent / "data" / "embeddings"
BATCH_SIZE = 128


class CsvDataset(Dataset):
    def __init__(self, csv_path):
        df = pd.read_csv(csv_path)
        self.sequences = df["sequence"].tolist()
        self.labels = df["label"].tolist()

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx]


tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
model = AutoModelForMaskedLM.from_pretrained(MODEL_NAME, trust_remote_code=True)

SELECTED_TASKS = ["promoter_all", "enhancers", "splice_sites_all", "H4K20me1", "H3K9me3"]

for task_dir in SELECTED_TASKS:
    print(f"Processing task: {task_dir}")
    for split in ["train", "val", "test"]:
        input_csv = PROCESSED_DIR / task_dir / f"{split}.csv"
        if not input_csv.exists():
            print(f"Нет файла {input_csv}")
            continue

        dataset = CsvDataset(input_csv)
        loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

        embeddings, labels = get_nt_embeddings(loader, tokenizer, model)

        out_task_dir = EMB_DIR / task_dir
        out_task_dir.mkdir(parents=True, exist_ok=True)

        df_emb = pd.DataFrame(embeddings, columns=[f"emb_{i}" for i in range(embeddings.shape[1])])
        df_emb["label"] = labels

        df_emb.to_csv(out_task_dir / f"{split}.csv", index=False)
        print(
            f"Число объектов в {split} выборке: {len(df_emb)}, уникальные классы: {set(df_emb['label'])}"
        )
        print(f"Сделано: {out_task_dir / f'{split}.csv'}")

Processing task: promoter_all


Generating embeddings: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 188/188 [00:28<00:00,  6.52it/s]


Число объектов в train выборке: 24000, уникальные классы: {0, 1}
Сделано: /cephfs/home/ledneva/Work/hw_mlops_itmo_2025/data/embeddings/promoter_all/train.csv


Generating embeddings: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 47/47 [00:07<00:00,  6.44it/s]


Число объектов в val выборке: 6000, уникальные классы: {0, 1}
Сделано: /cephfs/home/ledneva/Work/hw_mlops_itmo_2025/data/embeddings/promoter_all/val.csv


...

Число объектов в train выборке: 21950, уникальные классы: {0, 1}
Сделано: /cephfs/home/ledneva/Work/hw_mlops_itmo_2025/data/embeddings/H3K9me3/train.csv


Generating embeddings: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 43/43 [00:25<00:00,  1.70it/s]


Число объектов в val выборке: 5488, уникальные классы: {0, 1}
Сделано: /cephfs/home/ledneva/Work/hw_mlops_itmo_2025/data/embeddings/H3K9me3/val.csv


Generating embeddings: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:04<00:00,  1.75it/s]


Число объектов в test выборке: 850, уникальные классы: {0, 1}
Сделано: /cephfs/home/ledneva/Work/hw_mlops_itmo_2025/data/embeddings/H3K9me3/test.csv
